In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

/home/varad/snap/snapd-desktop-integration/83/Desktop/Falcon/falconllmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name =  "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.23s/it]


In [3]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Data

In [6]:
from datasets import load_dataset

train_data = load_dataset("json", data_files="../Dataset/data_train.json")
test_data = load_dataset("json", data_files="../Dataset/data_test.json")
# data = load_dataset("truthful_qa", "generation")

In [7]:
train_data

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'answer'],
        num_rows: 177
    })
})

In [8]:
test_data['train']

Dataset({
    features: ['index', 'question', 'answer'],
    num_rows: 43
})

In [9]:
train_dataset = train_data['train'].map(lambda x: {"input_text": x['question']  + "\n" + x['answer']})
tokenizer.pad_token = tokenizer.eos_token

In [10]:
train_encodings = tokenizer(train_dataset['input_text'], truncation=True, padding=True, max_length=256, return_tensors='pt')

In [11]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [12]:
train_dataset = TextDataset(train_encodings)

In [13]:
def generate(index):

  example_text = test_data['train'][index]['question']
  correct_answer = test_data['train'][index]['answer']
  tokenizer.pad_token = tokenizer.eos_token

  print("Question:")
  print(example_text)

  encoding = tokenizer(example_text, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=100, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)
  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Best Answer:")
  print(correct_answer)

  print()

In [14]:
index = 0
test_data['train'][index]['question']

'Which mode of transport is advisable for traveling during traffic hours?'

In [15]:
generate(index)

/home/varad/snap/snapd-desktop-integration/83/Desktop/Falcon/falconllmenv/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Which mode of transport is advisable for traveling during traffic hours?
Answer:
Which mode of transport is advisable for traveling during traffic hours?
It is advisable to use public transport or ride-sharing services during peak traffic hours to avoid traffic congestion and reduce the risk of accidents.
Best Answer:
The Delhi Metro is a reliable option for traveling during rush hours, as it's not affected by road traffic. Auto-rickshaws and taxis might get stuck in traffic.



In [16]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=80,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs-instruct",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/tmp/ipykernel_214705/365995831.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.108300
2,2.075600
3,2.068800
4,2.141600
5,2.073700
6,1.967400
7,2.163300
8,2.028400
9,2.036300
10,1.969600


TrainOutput(global_step=240, training_loss=0.558814515421788, metrics={'train_runtime': 976.9169, 'train_samples_per_second': 14.495, 'train_steps_per_second': 0.246, 'total_flos': 2.198795051741184e+16, 'train_loss': 0.558814515421788, 'epoch': 80.0})

In [17]:
model.save_pretrained("./outputs-instruct")

In [18]:

generate(index)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Which mode of transport is advisable for traveling during traffic hours?


/home/varad/snap/snapd-desktop-integration/83/Desktop/Falcon/falconllmenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Answer:
Which mode of transport is advisable for traveling during traffic hours?
For traveling during traffic hours, it's advisable to use public transport options like buses and metro stations. Taxis are recommended for avoiding traffic congestion. It's important to avoid driving and finding alternative transportation.
Best Answer:
The Delhi Metro is a reliable option for traveling during rush hours, as it's not affected by road traffic. Auto-rickshaws and taxis might get stuck in traffic.

